In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import ssl

In [22]:
#Descargamos por si las moscas
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dmuno\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dmuno\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dmuno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dmuno\AppData\Roaming\nltk_data...


True

In [23]:
# Vamos a descativar ssl
ssl._create_default_https_context = ssl._create_unverified_context
url= ""
data= pd.read_csv("https://raw.githubusercontent.com/dmunoz2019/spam/main/spam.csv", encoding="latin-1")
data.head(5)
print(data.columns)


Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')


In [24]:
#Vamos a elegir unicamente v1 y v2 porque no  hay elementos en las columnas sin nombre
data = data[['v1', 'v2']]
data.columns = ['label', 'message']
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [25]:
# Preprocesamiento utilizando NLTK
lemmatizer = WordNetLemmatizer()

In [26]:
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Tokenizar
    words = word_tokenize(text)
    # Eliminar palabras vacías
    words = [word for word in words if word not in stopwords.words('english')]
    # Lematizar
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

In [27]:
data['message'] = data['message'].apply(preprocess_text)

In [28]:
# Convertir a vectores numéricos con TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(data['message'])
y = data['label']

In [29]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
# Entrenar modelo Naive Bayes y ajustar hiperparámetros
params = {'alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0]}
nb_model = MultinomialNB()
grid_search = GridSearchCV(nb_model, param_grid=params, scoring='f1', cv=5)
grid_search.fit(X_train, y_train)

C:\Users\dmuno\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\dmuno\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\dmuno\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\dmuno\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_d

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0]},
             scoring='f1')

In [32]:
# Evaluación
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       965
        spam       0.91      0.91      0.91       150

    accuracy                           0.98      1115
   macro avg       0.95      0.95      0.95      1115
weighted avg       0.98      0.98      0.98      1115

[[951  14]
 [ 13 137]]


C:\Users\dmuno\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\dmuno\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\dmuno\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\dmuno\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_d

In [33]:
def predict_spam(message, model, vectorizer, preprocess_func):
    # Preprocesar el mensaje
    preprocessed_msg = preprocess_func(message)
    # Convertir el mensaje preprocesado en un vector numérico
    vectorized_msg = vectorizer.transform([preprocessed_msg])
    # Usar el modelo para predecir
    prediction = model.predict(vectorized_msg)
    return prediction[0]

# Definición de la función de preprocesamiento
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Tokenizar
    words = nltk.word_tokenize(text)
    # Eliminar palabras vacías
    words = [word for word in words if word not in nltk.corpus.stopwords.words('english')]
    # Lematizar
    words = [WordNetLemmatizer().lemmatize(word) for word in words]
    return ' '.join(words)



In [34]:

# Ejemplo de uso con un spam:
message = "SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info"
prediction = predict_spam(message, grid_search.best_estimator_, tfidf_vectorizer, preprocess_text)
print(f"Se clasifica este mensaje en: {prediction}")

The message is classified as: spam


In [35]:
# Ejemplo de uso con un correo normal:
message = "Anything lor... U decide..."
prediction = predict_spam(message, grid_search.best_estimator_, tfidf_vectorizer, preprocess_text)
print(f"Se clasifica este mensaje en: {prediction}")

Se clasifica este mensaje en: ham
